In [158]:
import json,re
import pandas as pd
import openai
from IPython.display import display, Markdown

In [113]:
training_df = pd.read_json("training_data_fixed.jsonl", lines=True)
training_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   prompt  15 non-null     object
 1   code    15 non-null     object
dtypes: object(2)
memory usage: 372.0+ bytes


In [147]:
def etract_components(prompt):
    pattern = r'''(.*)# Tools Available:(.*)# Memory:(.*)# Additional Context Gathered:(.*)# Conversation History \(Last .+\):(.*)# Additional Info:(.*)# Learning Plan:(.*)# Current Time:(.*)# Instructions:(.*)# Here is the last input from student.(.*)# Plan code:'''

    match = re.search(pattern, prompt, re.DOTALL)

    if match:
        preamble = match.group(1).strip()
        tools_available = match.group(2).strip()
        memory = match.group(3).strip()
        additional_context = match.group(4).strip()
        conversation_history = match.group(5).strip()
        additional_info = match.group(6).strip()
        learning_plan = match.group(7).strip()
        current_time = match.group(8).strip()
        instructions = match.group(9).strip()
        student_input = match.group(10).strip()
        # Create a dictionary with the extracted values
        data = {
            "preamble": preamble,
            "Tools Available": tools_available,
            "Memory": memory,
            "Additional Context Gathered": additional_context,
            "Conversation History (Last 10)": conversation_history,
            "Additional Info": additional_info,
            "Learning Plan": learning_plan,
            "Current Time": current_time,
            "Instructions": instructions,
            "Student Input": student_input
        }

        # Print the dictionary
        return data

    else:
        print(match)
        raise ValueError("regex not matching :")
# print(training_df.prompt.iloc[0])
etract_components(training_df.prompt.iloc[0])

{'preamble': 'You are an AI Teacher guiding a student toward their goals. Prepare customized plans, track their progress and skills, and personalize your responses using humor, challenges, and appreciation based on their personality ,goal and progreass.',
 'Tools Available': "- 'MemoryManager':\n\n    A class to manage in-memory short key-value pairs along with their update timestamps.\n    \n    \n    * `get_memory(key: str) -> str` \t# Retrieves the value associated with a given key from memory.\n    \n    * `get_memory_snapshot() -> str` \t# Retrieves a formatted snapshot of the memory, showing keys, values and the time elapsed since their last update.\n    \n    * `update_memory(key: str, value: str) -> None` \t# Updates the memory with a new short key-value pair. Make sure values are less than 200 characters.\n\n- 'MCQExaminer':\n\n    * `get_question(subject: str, difficulty: str) -> str` \t# Get an MCQ question based on the subject and difficulty level.\n    \n    * `grade_answe

In [152]:
prompt_components_df = pd.json_normalize(training_df.prompt.apply(etract_components))
components_df  = pd.concat([training_df, prompt_components_df], axis = 1)
components_df.head()

,prompt,code,preamble,Tools Available,Memory,Additional Context Gathered,Conversation History (Last 10),Additional Info,Learning Plan,Current Time,Instructions,Student Input
0,\nYou are an AI Teacher guiding a student towa...,"```python\nself.tools['Responder'].respond(""He...",You are an AI Teacher guiding a student toward...,- 'MemoryManager':\n\n A class to manage in...,,,-,None,No plan locked yet.,2024-09-03 16:55:33.860453,- Generate Only Python Code to call the approp...,Student: Hello
1,\nYou are an AI Teacher guiding a student towa...,```python\n\n# Update memory with learning goa...,You are an AI Teacher guiding a student toward...,- 'MemoryManager':\n\n A class to manage in...,,,- *student*: Hello\n\n- *Tutor*: Hello there! ...,None,No plan locked yet.,2024-09-03 16:55:53.272397,- Generate Only Python Code to call the approp...,Student: Ds algo
2,\nYou are an AI Teacher guiding a student towa...,```python\n# Lock the suggested learning plan\...,You are an AI Teacher guiding a student toward...,- 'MemoryManager':\n\n A class to manage in...,learning_goal: Ds algo (updated 2m 36s ago),,- *student*: Hello\n\n- *Tutor*: Hello there! ...,None,No plan locked yet.,2024-09-03 16:58:31.617221,- Generate Only Python Code to call the approp...,Student: Sure go ahead.
3,\nYou are an AI Teacher guiding a student towa...,"```python\nself.tools['Responder'].respond(""Aw...",You are an AI Teacher guiding a student toward...,- 'MemoryManager':\n\n A class to manage in...,learning_goal: Ds algo (updated 12m 13s ago)\n...,,- *student*: Hello\n\n- *Tutor*: Hello there! ...,None,"- **Learning Plan** (Importance: 1, Score: 0.0...",2024-09-03 17:08:08.092343,- Generate Only Python Code to call the approp...,"Student: Sure, let's dive into Arrays and Stri..."
4,\nYou are an AI Teacher guiding a student towa...,```python\n# Set the default programming langu...,You are an AI Teacher guiding a student toward...,- 'MemoryManager':\n\n A class to manage in...,learning_goal: Ds algo (updated 14m 40s ago)\n...,,- *student*: Hello\n\n- *Tutor*: Hello there! ...,None,"- **Learning Plan** (Importance: 1, Score: 0.0...",2024-09-03 17:10:35.564302,- Generate Only Python Code to call the approp...,"Student: Yeah, sure. Arrays and strings sound..."


In [159]:
display(Markdown(training_df['code'].iloc[0]))

```python
self.tools['Responder'].respond("Hello there! What subject or skill are you aiming to improve? Let's set some goals to get started.")
```


# Evaluation metric

1. fraction of Correct Method calls - jaccard distance
2. Edit distance
3. 

In [165]:
def list_levenshtein_distance(list1, list2):
    """
    Calculate the Levenshtein distance between two lists of words.

    :param list1: First list of words.
    :param list2: Second list of words.
    :return: The Levenshtein distance between the two lists.
    """
    m, n = len(list1), len(list2)
    dp = [[0] * (n + 1) for _ in range(m + 1)]

    # Initialize the distance matrix
    for i in range(m + 1):
        dp[i][0] = i
    for j in range(n + 1):
        dp[0][j] = j

    # Compute the distance matrix
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            cost = 0 if list1[i - 1] == list2[j - 1] else 1
            dp[i][j] = min(dp[i - 1][j] + 1,    # Deletion
                           dp[i][j - 1] + 1,    # Insertion
                           dp[i - 1][j - 1] + cost)  # Substitution

    return dp[m][n]

# Example usage
list1 = ['apple', 'banana', 'cherry']
list2 = ['apple', 'banana', 'banana']

print(list_levenshtein_distance(list1, list2))  # Output: edit distance between the two lists

def jaccard_similarity(list1, list2):
    """
    Calculate the Jaccard Similarity between two lists of words.

    :param list1: First list of words.
    :param list2: Second list of words.
    :return: The Jaccard Similarity between the two lists.
    """
    set1 = set(list1)
    set2 = set(list2)
    
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    
    return intersection / union if union != 0 else 0

def jaccard_distance(list1, list2):
    """
    Calculate the Jaccard Distance between two lists of words.

    :param list1: First list of words.
    :param list2: Second list of words.
    :return: The Jaccard Distance between the two lists.
    """
    return 1 - jaccard_similarity(list1, list2)



1


In [169]:
display(Markdown(training_df.code.iloc[1]))

def extract_tools_and_methods(code_string):
    """
    Extract tools and their methods from a code string.

    :param code_string: String containing the code with tool method calls.
    :return: List of tuples where each tuple contains the tool and its method.
    """
    # Regular expression to match tool and method calls
    pattern = re.compile(r"self\.tools\['(\w+)'\]\.(\w+)\(")
    
    # Find all matches
    matches = pattern.findall(code_string)
    
    # Create a list of tuples (tool, method)
    tools_and_methods = [(match[0], match[1]) for match in matches]
    
    return tools_and_methods

extract_tools_and_methods(training_df.code.iloc[1])

```python

# Update memory with learning goal
self.tools['MemoryManager'].update_memory('goal', 'Improve DS Algo')

# Suggest a customized learning plan for the goal
learning_plan = self.tools['LearningPlanManager'].suggest_learning_plan(goal='Improve DS Algo')

# Provide the suggested learning plan to the user for confirmation
self.tools['Responder'].respond(f"Great! Let's dive into Data Structures and Algorithms. Here's a plan I suggest:

{learning_plan}

Shall we lock this plan and get started?")

```


[('MemoryManager', 'update_memory'),
 ('LearningPlanManager', 'suggest_learning_plan'),
 ('Responder', 'respond')]

In [ ]:
def evaluate(gt, pred):
    gt_tools_methods = extract_tools_and_methods(gt)
    pred_tools_methods = extract_tools_and_methods(pred)
    return {
        'jaccard_distance' : jaccard_distance(gt_tools_methods, pred_tools_methods),
        'edit_distance': 
    }

# Finetune openAI
